In [1]:
!pip install tqdm==4.64.1
!pip install dlib
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=a222949230ef4d2ceb1ea43c982c4eaa8abb0e769e4f21e9929391716db9e699
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
import cv2
import os
import face_recognition
import numpy as np
from tqdm import tqdm
import pandas as pd
import json

In [10]:
# split video into frames

video = cv2.VideoCapture('bee.mp4')

fps = video.get(cv2.CAP_PROP_FPS)

num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

!mkdir frames

for i in tqdm(range(num_frames)):

    ret, frame = video.read()

    if not ret:
        break
    if not os.path.exists(f'frames/frame_{i}.jpg'):
      cv2.imwrite(f'frames/frame_{i}.jpg', frame)

video.release()


mkdir: cannot create directory ‘frames’: File exists


100%|██████████| 131664/131664 [15:41<00:00, 139.78it/s]


In [ ]:
# then find landmarks and using face_detection
# im going to split it into chunks 0-20000, 20000-40000, 40000-60000, 60000-80000, 80000-100000, 100000-120000, 120000-131664
# hard coded

!mkdir cropped

def crop(points):
  x_min = min(point[0] for point in points)
  x_max = max(point[0] for point in points)
  y_min = min(point[1] for point in points)
  y_max = max(point[1] for point in points)
  return x_min, y_min, x_max, y_max

frame_count = 131664

for i in tqdm(range(0, 20000)):
  num = str(i+1)
  for _ in range(len(str(frame_count))-len(num)):
    num = '0'+ num
  frame = cv2.imread(f"frames/frame_{i}.jpg")
  face_landmarks_list = face_recognition.face_landmarks(frame)
  if face_landmarks_list != []:
    [x, y, w, h] = crop(face_landmarks_list[0]['top_lip'] + face_landmarks_list[0]['bottom_lip'])
    frame = frame[y:h, x:w]
  cv2.imwrite(f"cropped/{num}.jpg", frame)




mkdir: cannot create directory ‘cropped’: File exists


  5%|▌         | 1018/20000 [03:07<1:06:02,  4.79it/s]

In [5]:
# then align each frame a with phoneme
align = json.load(open("align.json"))['words']
aligned = []

for _d in range(len(align)):
  d = align[_d]
  if d["case"] == "success":
    if _d > 0 and align[_d-1]["case"] == "success" and d["start"] - align[_d-1]["end"] != 0:
      aligned.append({"duration":d["start"] - align[_d-1]["end"],"phone":"none","start":align[_d-1]["end"],"end":d["start"],"case":"success"})
    phones = d["phones"]
    for p in range(len(phones)):
      if p == 0:
        phones[p]["start"] = d["start"]
        phones[p]["end"] = phones[p]["start"] + phones[p]["duration"]
      else:
        phones[p]["start"] = phones[p-1]["end"]
        phones[p]["end"] = phones[p]["start"] + phones[p]["duration"]
      phones[p]["case"] = "success"
      phones[p]["phone"] = phones[p]["phone"].split("_")[0] 
      phones[p]["duration"] = phones[p]["end"] - phones[p]["start"]
      if phones[p]["phone"] == "oov":
        phones[p]["case"] = "failed"
        phones[p]["phone"] = "none"
    for i in phones:
      aligned.append(i)
  elif align[_d]["case"] != "success" and align[_d-1]["case"] == "success" and _d != 0:
    index = _d
    while True:
      index += 1
      if align[index]["case"] == "success":
        aligned.append({"duration":align[index]["start"] - align[_d-1]["end"],"phone":"none","start":align[_d-1]["end"],"end":align[index]["start"],"case":"failed"})
        break
  elif align[_d]["case"] != "success" and _d == 0:      
    index = _d
    while True:
      index += 1
      if align[index]["case"] == "success":
        aligned.append({"duration":align[index]["start"],"phone":"none","start":0,"end":align[index]["start"],"case":"failed"})
        break

pd.DataFrame(aligned)

0
1
